# Semantic Kernel OpenBnB MCP -palvelinintegraation kanssa

Tämä muistikirja havainnollistaa, kuinka Semantic Kernel toimii yhdessä varsinaisen OpenBnB MCP -palvelimen kanssa etsiessä todellisia Airbnb-majoituksia käyttämällä MCPStdioPluginia. LLM-käyttöoikeutta varten se käyttää Microsoft Foundrya. Voit määrittää ympäristömuuttujasi seuraamalla [Asennusoppituntia](/00-course-setup/README.md).


## Tuo tarvitut paketit


In [ ]:
# Import cell - Updated imports
import json
import os
import asyncio
import subprocess
import sys


from dotenv import load_dotenv
from IPython.display import display, HTML
from typing import Annotated

from semantic_kernel.agents import ChatCompletionAgent, ChatHistoryAgentThread
from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion
from semantic_kernel.connectors.mcp import MCPStdioPlugin
from semantic_kernel.contents import FunctionCallContent, FunctionResultContent, StreamingTextContent

## MCP-laajennusyhteyden luominen

Yhdistämme [OpenBnB MCP -palvelimeen](https://github.com/openbnb-org/mcp-server-airbnb) käyttäen MCPStdioPluginia. Tämä palvelin tarjoaa Airbnb-hakutoiminnon @openbnb/mcp-server-airbnb -paketin kautta.


## Asiakkaan luominen

Tässä esimerkissä käytämme Microsoft Foundrya LLM-yhteyttä varten. Varmista, että ympäristömuuttujasi on asetettu oikein.


## Ympäristöasetukset

Määritä Azure OpenAI -asetukset. Varmista, että sinulla on seuraavat ympäristömuuttujat asetettuna:
- `AZURE_OPENAI_CHAT_DEPLOYMENT_NAME`
- `AZURE_OPENAI_ENDPOINT`
- `AZURE_OPENAI_API_KEY`


In [ ]:
# Creating the Client cell - Updated for Azure
load_dotenv()

# Azure OpenAI configuration
# Ensure these environment variables are set:
# - AZURE_OPENAI_CHAT_DEPLOYMENT_NAME
# - AZURE_OPENAI_ENDPOINT
# - AZURE_OPENAI_API_KEY (optional if using DefaultAzureCredential)

chat_completion_service = AzureChatCompletion(
    deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
    endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    # Optional - will use DefaultAzureCredential if not set
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
)

## OpenBnB MCP -integraation ymmärtäminen

Tämä muistikirja yhdistää **todelliseen OpenBnB MCP -palvelimeen**, joka tarjoaa aitoa Airbnb-hakutoiminnallisuutta.

### Miten se toimii:

1. **MCPStdioPlugin**: Käyttää standardisyötön/ -tulostuksen kommunikointia MCP-palvelimen kanssa  
2. **Aito NPM-paketti**: Lataa ja suorittaa `@openbnb/mcp-server-airbnb` npx:n kautta  
3. **Elävät tiedot**: Palauttaa todellisia Airbnb-kohdetietoja niiden API:ista  
4. **Toimintojen löytäminen**: Agentti löytää automaattisesti MCP-palvelimelta käytettävissä olevat toiminnot

### Käytettävissä olevat toiminnot:

OpenBnB MCP -palvelin tarjoaa tyypillisesti:  
- **search_listings** - Hae Airbnb-kohteita sijainnin ja kriteerien perusteella  
- **get_listing_details** - Hae yksityiskohtaiset tiedot tietyistä kohteista  
- **check_availability** - Tarkista saatavuus tiettyinä päivinä  
- **get_reviews** - Hae kohteiden arvosteluja  
- **get_host_info** - Hae tietoja kohteen isännästä

### Vaatimukset:

- **Node.js** asennettuna järjestelmääsi  
- **Internet-yhteys**, jotta MCP-palvelinpakkauksen lataaminen onnistuu  
- **NPX** käytettävissä (sisältyy Node.js:ään)

### Yhteyden testaaminen:

Voit testata MCP-palvelinta manuaalisesti suorittamalla:  
```bash
npx -y @openbnb/mcp-server-airbnb
```
  
Tämä lataa ja käynnistää OpenBnB MCP -palvelimen, johon Semantic Kernel sitten yhdistää saadakseen oikeat Airbnb-tiedot.


## Agentin suorittaminen OpenBnB MCP -palvelimen kanssa

Nyt suoritetaan AI-agentti, joka yhdistää OpenBnB MCP -palvelimeen etsiäkseen todellisia Airbnb-majoituksia Tukholmasta 2 aikuiselle ja 1 lapselle. Voit vapaasti muuttaa `user_inputs`-listaa muuttaaksesi hakuehtoja.


In [ ]:
user_inputs = [
    "Find Airbnb in Stockholm for 2 adults 1 kid",
]


async def main():
    """Main function to run the MCP-enabled agent with real OpenBnB server using Azure OpenAI"""

    try:
        print("🚀 Starting with Azure OpenAI...")
        
        # Verify environment variables
        print("🔍 Checking Azure environment variables...")
        required_vars = ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME", "AZURE_OPENAI_ENDPOINT", "AZURE_OPENAI_API_KEY"]
        for var in required_vars:
            if os.getenv(var):
                print(f"✅ {var} is set")
            else:
                print(f"❌ {var} is NOT set")
        
        print("\n🔧 Creating MCP Plugin...")
        
        # Create MCP plugin connection to real OpenBnB server
        # Based on the GitHub repo, the server doesn't need special env vars
        async with MCPStdioPlugin(
            name="AirbnbSearch",
            description="Search for Airbnb accommodations using OpenBnB MCP server",
            command="npx",
            args=["-y", "@openbnb/mcp-server-airbnb"],
        ) as airbnb_plugin:

            print("✅ MCP Plugin created and connected")
            
            # Wait a moment for the server to fully initialize
            await asyncio.sleep(2)
            
            # Try to list available tools
            try:
                tools = await airbnb_plugin.get_tools()
                print(f"🔧 Available tools: {[tool.name for tool in tools]}")
            except Exception as e:
                print(f"⚠️ Could not list tools: {str(e)}")

            # Create the Azure OpenAI service with proper configuration
            print("\n🤖 Creating Azure OpenAI service...")
            service = AzureChatCompletion(
                deployment_name=os.getenv("AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"),
                endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
                api_key=os.getenv("AZURE_OPENAI_API_KEY"),
            )
            
            # Create agent with the service instance
            agent = ChatCompletionAgent(
                service=service,
                name="AirbnbAgent",
                instructions="""You are an Airbnb search assistant. Use the available functions to search for properties. 
                Format results in a clear HTML table with columns for property name, price, rating, and link.""",
                plugins=[airbnb_plugin],
            )

            print("✅ Agent created with Azure OpenAI")

            # Process each user input
            thread: ChatHistoryAgentThread | None = None

            for user_input in user_inputs:
                print(f"\n🔍 User: {user_input}")
                
                try:
                    # Use the simpler get_response method
                    response = await agent.get_response(messages=user_input, thread=thread)
                    thread = response.thread
                    
                    # Process the response text
                    response_text = str(response)
                    
                    # Remove any markdown code blocks around HTML
                    response_text = response_text.replace('```html', '').replace('```', '')
                    
                    # Display the result
                    print(f"🤖 {response.name}: {response_text[:200]}..." if len(response_text) > 200 else response_text)
                    
                    # If response contains HTML table, display it properly
                    if '<table' in response_text.lower():
                        # Add CSS styling for better table rendering
                        table_css = """
                        <style>
                            .airbnb-results table {
                                border-collapse: collapse;
                                width: 100%;
                                margin: 10px 0;
                            }
                            .airbnb-results th, .airbnb-results td {
                                border: 1px solid #ddd;
                                padding: 8px;
                                text-align: left;
                            }
                            .airbnb-results th {
                                background-color: #f2f2f2;
                                font-weight: bold;
                            }
                            .airbnb-results tr:nth-child(even) {
                                background-color: #f9f9f9;
                            }
                            .airbnb-results a {
                                color: #1976d2;
                                text-decoration: none;
                            }
                            .airbnb-results a:hover {
                                text-decoration: underline;
                            }
                        </style>
                        """
                        html_output = f'{table_css}<div class="airbnb-results">{response_text}</div>'
                        display(HTML(html_output))
                    else:
                        # Display as regular text if no table
                        display(HTML(f'<div class="airbnb-results">{response_text}</div>'))
                        
                except Exception as e:
                    print(f"❌ Error processing user input: {str(e)}")
                    import traceback
                    traceback.print_exc()
                
            # Cleanup
            if thread:
                await thread.delete()
                print("🧹 Thread cleaned up")
                
    except Exception as e:
        print(f"❌ Main error: {str(e)}")
        import traceback
        traceback.print_exc()

# Run the main function
print("🚀 Starting MCP Agent...")
await main()
print("✅ Done!")

# Yhteenveto
Onnittelut! Olet onnistuneesti rakentanut tekoälyagentin, joka integroituu todellisen maailman majoituksen hakuun Model Context Protocolin (MCP) avulla:

## Käytetyt teknologiat:
- Semantic Kernel – Älykkäiden agenttien rakentamiseen Azure OpenAI:lla
- Microsoft Foundry – LLM-ominaisuuksiin ja chat-kompletioon
- MCP (Model Context Protocol) – Standardoitu työkalujen integrointi
- OpenBnB MCP Server – Todellisen Airbnb-haun toteuttamiseen
- Node.js/NPX – Ulkoisen MCP-palvelimen ajamiseen

## Mitä olet oppinut:
- MCP-integraatio: Semantic Kernel -agenttien yhdistäminen ulkoisiin MCP-palvelimiin
- Reaaliaikainen tiedonhaku: Todellisten Airbnb-kohteiden hakeminen live-rajapintojen kautta
- Protokollaviestintä: stdio-viestinnän käyttö agentin ja MCP-palvelimen välillä
- Funktioiden löytäminen: Saatavilla olevien funktioiden automaattinen löytäminen MCP-palvelimilta
- Suoratoistovastaukset: Funktiokutsujen kaappaus ja lokitus reaaliajassa
- HTML-renderöinti: Agentin vastausten muotoilu tyylitellyillä taulukoilla ja interaktiivisilla näytöillä

## Seuraavat askeleet:
- Integroi lisä MCP-palvelimia (sää, lennot, ravintolat)
- Rakenna moni-agenttijärjestelmä yhdistäen MCP- ja A2A-protokollat
- Luo omia MCP-palvelimia omia datasoureja varten
- Toteuta pysyvä keskustelumuisti istuntojen välillä
- Ota agentti käyttöön Azure Functions -ympäristössä MCP-palvelinohjausten kanssa
- Lisää käyttäjän todennus ja varausominaisuudet


---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
**Vastuuvapauslauseke**:
Tämä asiakirja on käännetty tekoälykäännöspalvelulla [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, otathan huomioon, että automaattiseen käännökseen saattaa sisältyä virheitä tai epätarkkuuksia. Alkuperäinen asiakirja omalla kielellään on pätevä lähde. Tärkeissä asioissa suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa tämän käännöksen käytöstä aiheutuvista väärinymmärryksistä tai tulkinnoista.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
